In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mne
from scipy import signal
from scipy.signal import butter, filtfilt, iirnotch, detrend
from definitions import CHAN_LOC
import pyxdf  # For loading .xdf files

In [3]:
def load_xdf_file(filepath):
    """Load the XDF file using pyxdf."""
    streams, header = pyxdf.load_xdf(filepath)
    return streams, header

In [4]:
# def perform_ICA(streams):
#     if streams['time_series'].shape
#         # Attempt to load EEG data from the primary stream
#         data = streams[0]['time_series'][:,:9]
#         sampling_rate = streams[0]['info']['effective_srate']
#     except Exception as e:
#         print(f"Primary stream loading failed: {e}. Trying secondary stream...")
#         # Load EEG data from the alternative stream if the first attempt fails
#         data = streams[1]['time_series'][:9, :]
#         sampling_rate = streams[1]['info']['effective_srate']

#     # Load your channel locations file
#     chanlocs = r"C:\Users\Ferran\Desktop\TFM\TFM_EEG\Documents\Standard-10-20-Cap10.txt"

#     # Create an MNE RawArray object from the data
#     num_channels = data.shape[0]
#     ch_names = ['EEG' + str(i) for i in range(1, num_channels + 1)]  # Assuming generic channel names
#     montage = mne.channels.read_custom_montage(chanlocs)  # Load channel locations
#     info = mne.create_info(ch_names=ch_names, sfreq=sampling_rate, ch_types='eeg')
#     raw = mne.io.RawArray(data, info)
#     raw.set_montage(montage)
    
#     # Preprocess the data (bandpass filter between 1-50 Hz)
#     raw.filter(1., 50., fir_design='firwin')
    
#     # Run ICA
#     ica = mne.preprocessing.ICA(n_components=num_channels, method='fastica', random_state=97)
#     ica.fit(raw)
    
#     # Optional: Save the ICA results
#     ica.save('yourdata_ica.fif')  # Uncomment and modify the path to save the ICA
    
#     return raw, ica


In [5]:
import numpy as np
import mne

def perform_ICA(streams, side_identifier):
    """
    Perform ICA on EEG data and save the ICA solution with a unique filename.

    Parameters:
    streams : list
        List containing the EEG data (as dictionaries).
    side_identifier : int
        1 for Right (R) side, 2 for Left (L) side, used to differentiate the filenames.

    Returns:
    raw : mne.io.Raw
        The raw EEG data after preprocessing.
    ica : mne.preprocessing.ICA
        The fitted ICA solution.
    """
    # Check the structure of the streams
    print(f"Length of the streams list: {len(streams)}")

    # Initialize variables
    data = None
    sampling_rate = None
    
    # Convert the time_series to a NumPy array and check the first stream
    first_stream_data = np.array(streams[0]['time_series'])
    second_stream_data = np.array(streams[1]['time_series']) if len(streams) > 1 else None

    # Check if the first stream has 10 channels
    if first_stream_data.shape[0] == 10:  
        data = first_stream_data
        sampling_rate = streams[0]['info']['effective_srate']
        print("Using first stream with shape:", data.shape)
    elif second_stream_data is not None:
        # If the first stream does not have 10 channels, check the second stream
        data = second_stream_data
        sampling_rate = streams[1]['info']['effective_srate']
        print("Using second stream with shape:", data.shape)
    else:
        raise ValueError("No valid data found in the streams.")

    # Ensure we have valid data to work with
    if not isinstance(data, np.ndarray) or not np.issubdtype(data.dtype, np.number):
        raise ValueError("No valid numeric data found in any of the streams.")

    # Create a RawArray object for MNE
    n_channels = data.shape[1]  # Number of channels is the number of columns (10)
    n_samples = data.shape[0]    # Number of samples is the number of rows (e.g., 47232)

    # Create the correct channel names (for example)
    ch_names = ['F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'GND']
    info = mne.create_info(ch_names=ch_names, sfreq=sampling_rate, ch_types='eeg')

    # Create the RawArray object with the correct shape
    raw = mne.io.RawArray(data.T, info)  # Transpose data to fit MNE format (channels x samples)

    # Set montage (if applicable)
    montage = mne.channels.make_dig_montage(ch_pos={
        'F3': (-39, 0.33333, 0), 
        'Fz': (0, 0.25556, 0),
        'F4': (39, 0.33333, 0),
        'C3': (-90, 0.25556, 0),
        'Cz': (90, 0, 0),
        'C4': (90, 0.25556, 0),
        'P3': (-141, 0.33333, 0),
        'Pz': (180, 0.25556, 0),
        'P4': (141, 0.33333, 0),
        'GND': (-162, 0.51111, 0)
    })
    
    raw.set_montage(montage)

    # High-pass filter the data for ICA performance
    raw.filter(l_freq=1.0, h_freq=None)

    # Perform ICA with the number of components equal to the number of channels
    ica = mne.preprocessing.ICA(n_components=n_channels, random_state=97, max_iter=800)
    ica.fit(raw)

    # Save the ICA solution with a unique filename
    ica_filename = f"ICA_solution_side_{side_identifier}.fif"
    try:
        ica.save(ica_filename)
        print(f"ICA solution saved to {ica_filename}")
    except Exception as e:
        print(f"Error saving ICA solution: {e}")

    return raw, ica


In [6]:
def filtering(EEG_data):
    # Sampling rate
    sampling_rate = EEG_data.info['sfreq']

    # Butterworth bandpass filter between 0.3 and 10 Hz (4th order)
    b_butter, a_butter = butter(4, [0.3, 10], btype='bandpass', fs=sampling_rate)

    # Mu wave frequency range
    mu_range = [8, 12]
    b_mu, a_mu = butter(4, mu_range, btype='bandpass', fs=sampling_rate)

    # Notch filter for power line noise at 50 Hz
    b_notch, a_notch = iirnotch(50, 50 / 35, sampling_rate)

    # Initialize the filtered data
    eeg_data_notch = np.zeros_like(EEG_data.get_data())
    eeg_data_but = np.zeros_like(EEG_data.get_data())
    eeg_data_mu = np.zeros_like(EEG_data.get_data())
    eeg_data_mean = np.zeros_like(EEG_data.get_data())

    # Apply filtering to each channel
    for i in range(EEG_data.get_data().shape[0]):
        # Apply notch filter to remove 50 Hz power line noise
        eeg_data_notch[i, :] = filtfilt(b_notch, a_notch, EEG_data.get_data()[i, :])

        # Apply Butterworth bandpass filter (0.3-10 Hz)
        eeg_data_but[i, :] = filtfilt(b_butter, a_butter, detrend(eeg_data_notch[i, :]))

        # Apply Butterworth filter for mu waves (8-12 Hz)
        eeg_data_mu[i, :] = filtfilt(b_mu, a_mu, detrend(eeg_data_but[i, :]))

        # Calculate the mean for each channel after mu wave filtering
        eeg_data_mean[i, :] = np.mean(eeg_data_mu[i, :])

    # Transpose the filtered data
    eeg_data_but = eeg_data_but.T

    # Zero-mean the data
    channels_mean = np.mean(eeg_data_but, axis=1)
    channels_mean_matrix = np.tile(channels_mean[:, np.newaxis], (1, eeg_data_but.shape[1]))
    EEG_filt = eeg_data_but - channels_mean_matrix

    return EEG_filt

In [7]:
def plot_eeg(EEG_filt, events, sampling_rate, title):
    """ Plot EEG data with event markers. """
    time_vector = np.arange(len(EEG_filt)) / sampling_rate
    plt.figure()
    plt.plot(time_vector, EEG_filt[:, 5])  # Cz channel (index 5)
    plt.xlabel('Time (s)')
    plt.ylabel('EEG Data')
    plt.title(title)
    for event in events:
        plt.axvline(x=event / sampling_rate, color='r', linestyle='--')
    plt.show()

In [8]:
def plot_topoplot(mean_data, title):
    """ Plot the topographic map (topoplot) of EEG data. """
    mne.viz.plot_topomap(mean_data, pos=CHAN_LOC, show=True)
    plt.title(title)
    plt.show()


In [13]:
# Load XDF files
xdfFilePath_Right = r"C:\Users\Ferran\Desktop\TFM\TFM_EEG\DATASET\SUB_2\RIGHT\sub-P001\ses-S003\eeg\sub-P001_ses-S003_task-Default_run-001_eeg.xdf"
xdfFilePath_Left = r"C:\Users\Ferran\Desktop\TFM\TFM_EEG\DATASET\SUB_2\LEFT\sub-P001\ses-S004\eeg\sub-P001_ses-S004_task-Default_run-001_eeg.xdf"

streams_R, header_R = load_xdf_file(xdfFilePath_Right)
streams_L, header_L = load_xdf_file(xdfFilePath_Left)


In [16]:
for i, stream in enumerate(streams_L):
    print(f"Stream {i}: Keys = {stream.keys()}")


Stream 0: Keys = dict_keys(['info', 'footer', 'time_series', 'time_stamps', 'clock_times', 'clock_values'])
Stream 1: Keys = dict_keys(['info', 'footer', 'time_series', 'time_stamps', 'clock_times', 'clock_values'])


In [17]:
# Perform ICA
EEG_DATA_R, ICA_R = perform_ICA(streams_R,1)
EEG_DATA_L, ICA_L = perform_ICA(streams_L,2)

Length of the streams list: 2
Using second stream with shape: (47232, 10)
Creating RawArray with float64 data, n_channels=10, n_times=47232
    Range : 0 ... 47231 =      0.000 ...   184.507 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 845 samples (3.301 s)

Fitting ICA to data using 10 channels (please be patient, this may take a while)
Selecting by number: 10 components


C:\Users\Ferran\AppData\Local\Temp\ipykernel_31316\4900453.py:73: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


Fitting ICA took 0.2s.
Writing ICA solution to C:\Users\Ferran\Desktop\TFM\TFM_EEG\TESTS\ICA_solution_side_1.fif...
ICA solution saved to ICA_solution_side_1.fif
Length of the streams list: 2
Using second stream with shape: (40, 1)


C:\Users\Ferran\AppData\Local\Temp\ipykernel_31316\4900453.py:85: RuntimeWarning: This filename (ICA_solution_side_1.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica.save(ica_filename)


ValueError: No valid numeric data found in any of the streams.

In [18]:
import mne

# Specify the path to your .fif file
fif_file_path = r"C:\Users\Ferran\Desktop\TFM\TFM_EEG\TESTS\ICA_solution_side_1.fif"  # Replace with your actual file path

# Load the FIF file
raw = mne.io.read_raw_fif(fif_file_path, preload=True)

# Print information about the raw data
print(raw.info)

# Optionally, plot the data
raw.plot()


Opening raw data file C:\Users\Ferran\Desktop\TFM\TFM_EEG\TESTS\ICA_solution_side_1.fif...


C:\Users\Ferran\AppData\Local\Temp\ipykernel_31316\4190640621.py:7: RuntimeWarning: This filename (C:\Users\Ferran\Desktop\TFM\TFM_EEG\TESTS\ICA_solution_side_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_file_path, preload=True)


ValueError: No raw data in C:\Users\Ferran\Desktop\TFM\TFM_EEG\TESTS\ICA_solution_side_1.fif

In [ ]:
# Filter EEG data
EEG_filt_R = filtering(EEG_DATA_R)
EEG_filt_L = filtering(EEG_DATA_L)

# Plot EEG Data with Event Markers
sampling_rate = EEG_DATA_R.info['sfreq']
events_right = streams_R[0]['time_stamps'] * sampling_rate
events_left = streams_L[0]['time_stamps'] * sampling_rate

plot_eeg(EEG_filt_R, events_right, sampling_rate, 'EEG Data with Event Markers (Right)')
plot_eeg(EEG_filt_L, events_left, sampling_rate, 'EEG Data with Event Markers (Left)')

# Create and plot Topoplots
mean_R_data = np.mean(EEG_filt_R, axis=0)
mean_L_data = np.mean(EEG_filt_L, axis=0)

plot_topoplot(mean_R_data, 'EEG TOPOPLOT for Right Events')
plot_topoplot(mean_L_data, 'EEG TOPOPLOT for Left Events')